Jupyter notebook for analyzing how concatenated v. RMG built mechanisms compare

In [5]:
import os 
import re
import matplotlib.pyplot as plt
import rmgpy.chemkin
import numpy as np
import cantera as ct
import random
import pandas as pd

In [6]:
def load_chemkin_file_RMG(blend_folder): 
    """ 
    Load Chemkin file and extract the reactions and species. Must provide blend folder name.
    """
    
    full_path = os.path.join('/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core_redo',blend_folder,'chemkin')
    chemkin_path = os.path.join(full_path,'copies', 'copy_2_chem.inp')
   # chemkin_path = os.path.join(full_path,'chem_annotated.inp')
    dictionary_path = os.path.join(full_path,'species_dictionary.txt')
    transport_path = os.path.join(full_path,'tran.dat')


    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path)

    return species_list, reaction_list

In [7]:
def load_chemkin_file_concatenated(concatenated_folder): 
    """ 
    Load Chemkin file and extract the reactions and species.
    Must provide concatenated folder name. 
    """
    full_path = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/concatenate_equal_parts/{concatenated_folder}/concatenated/'
    chemkin_path = os.path.join(full_path,'copies', 'copy_chem.inp')
   # chemkin_path = os.path.join(full_path,'chem_annotated.inp')
    dictionary_path = os.path.join(full_path,'species_dictionary.txt')
    transport_path = os.path.join(full_path,'tran.dat')


    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path)

    return species_list, reaction_list

In [8]:
def compare_rxns(model_1_reactions, model_2_reactions): 
    '''
    Compares the reaction equations of each model. Uses to_cantera() on RMG model to eliminate chemkin_identifier 
    '''

################### mostly taken from diffmodel.py, compare_model_reactions() ###################

    #remove reactions with unknown species
    to_remove = []
    for reactionList in (model_1_reactions, model_2_reactions):
        for reaction in reactionList:
            for side in (reaction.products, reaction.reactants):
                for species in side:
                    if not species.molecule:
                        to_remove.append((reactionList, reaction))
                        print("Removing reaction {!r} that had unidentified species {!r}".format(reaction, species))
                        break
    if len(to_remove)!=0:
        for reactionList, reaction in to_remove:
            reactionList.remove(reaction)
        print(f'Following reactions removed because they have unknown species: {to_remove}')
    
    #find the common reactions and the unique reactions
    common_reactions = []
    common_reactions_equations = []
    unique_reactions_1 = []
    unique_reactions_2 = []
    copy_of_model_1_reactions = model_1_reactions.copy() #list 1
    copy_of_model_2_reactions = model_2_reactions.copy() #list 2

    
    for model_2_rxn in copy_of_model_2_reactions:
        for model_1_rxn in copy_of_model_1_reactions: # make a copy so you don't remove from the list you are iterating over
            counter = 0 #use this to see how many times this loop below was executed
            if model_2_rxn.is_isomorphic(model_1_rxn):
                common_reactions.append([model_1_rxn, model_2_rxn])
                common_reactions_equations.append([str(model_1_rxn), str(model_2_rxn)])
                copy_of_model_1_reactions.remove(model_1_rxn)
                counter += 1
            if counter>1:
                print([str(model_1_rxn),str(model_2_rxn)]) #let's see if a rxn has two matches
                
 
    #find the unique reactions of each model
    for model_2_rxn in copy_of_model_2_reactions:
        for r1, r2 in common_reactions:
            if model_2_rxn is r2:
            #if model_2_rxn.is_isomorphic(r2):
                break
        else:
            unique_reactions_2.append(model_2_rxn)
    for model_1_rxn in copy_of_model_1_reactions:
        for r1, r2 in common_reactions:
            if model_1_rxn is r1:
#             if model_1_rxn.is_isomorphic(r1):
                break
        else:
            unique_reactions_1.append(model_1_rxn)
            
    
    #find the reactions that have different kinetics (aren't identical and aren't similar)
    
    different_kinetics_reactions = []
    identical_kinetics_reactions = []
    similar_kinetics_reactions = []
    
    for r1, r2 in common_reactions: 
        if (r2.kinetics.is_identical_to(r1.kinetics)==False) and (r2.kinetics.is_similar_to(r1.kinetics)==False):
            different_kinetics_reactions.append((r1, r2))
        if r2.kinetics.is_identical_to(r1.kinetics)==True:
            identical_kinetics_reactions.append((r1, r2))
        if r2.kinetics.is_similar_to(r1.kinetics)==True:
            similar_kinetics_reactions.append((r1, r2))
            
            
    return similar_kinetics_reactions, identical_kinetics_reactions, different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactions_1, unique_reactions_2
    
    

In [10]:
def compare_species(model_1_species_list, model_2_species_list):
    """ Compares species list between models. """
    
    #two lists of same species, just have different names in each model
    common_species_1 = []
    common_species_2 = []
    
    #combined list of common species, with tuples of the two different names
    common_species = []
    
    for spc_1 in model_1_species_list:
        for spc_2 in model_2_species_list:
            if spc_1.is_isomorphic(spc_2):
                common_species_1.append(spc_1)
                common_species_2.append(spc_2)
                common_species.append((spc_1, spc_2))
                
                
    unique_species_1 = [spc_1 for spc_1 in model_1_species_list if spc_1 not in common_species_1]
    unique_species_2 = [spc_2 for spc_2 in model_2_species_list if spc_2 not in common_species_2]

                
    return common_species, unique_species_1, unique_species_2
    

In [11]:
#model 1 = concatenated, model 2 = rmgbuilt

RMG_species_list, RMG_reaction_list = load_chemkin_file_RMG('CH2F2_CH3F')
con_species_list, con_reaction_list = load_chemkin_file_concatenated('CH2F2_CH3F')

In [12]:
common_species, unique_species_conc, unique_species_rmg = compare_species(con_species_list, RMG_species_list)

In [13]:

similar_kinetics_reactions, identical_kinetics_reactions, different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactions_conc, unique_reactions_rmg = compare_rxns(con_reaction_list, RMG_reaction_list) 

In [14]:
################ #ANALYSIS OF MODELS

print(f"RMG mechanism has {len(RMG_reaction_list)} reactions and {len(RMG_species_list)} species.")
print('\n')
print(f"Concatenated mechanism reaction has {len(con_reaction_list)} reactions and {len(con_species_list)} species.")
print('\n')
print(f"common reactions: {len(common_reactions)}")
print('\n')
print(f'\tOf common reactions, {len(different_kinetics_reactions)} reactions have different kinetics')
print('\n')
print(f'\tOf common reactions, {len(identical_kinetics_reactions)} reactions have identical kinetics')
print('\n')
print(f'\tOf common reactions, {len(similar_kinetics_reactions)-len(identical_kinetics_reactions)} reactions have similar kinetics')
print('\n')
print(f'common species: {len(common_species)}')
print('\n')
print(f'unique species in Conc. model: {len(unique_species_conc)}')
print('\n')
print(f'unique species in RMG model: {len(unique_species_rmg)}')


#NOTE: concatenated models can be greatly reduced in RMG bc you get the same flamespeed with much less reactions.
# Also include data about the original mechanisms and how they compare to conc v. rmg built. 

RMG mechanism has 3367 reactions and 141 species.


Concatenated mechanism reaction has 2157 reactions and 118 species.


common reactions: 2000


	Of common reactions, 208 reactions have different kinetics


	Of common reactions, 1340 reactions have identical kinetics


	Of common reactions, 452 reactions have similar kinetics


common species: 118


unique species in Conc. model: 0


unique species in RMG model: 23


In [15]:
#comparing single RMG models of CH2F2 and CH3F
    
def load_chemkin_file_RMG(blend_folder): 
    """ 
    Load Chemkin file and extract the reactions and species. Must provide blend folder name.
    """
    
    full_path = os.path.join('/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/',blend_folder,'chemkin')
    chemkin_path = os.path.join(full_path,'chem_annotated.inp')
    dictionary_path = os.path.join(full_path,'species_dictionary.txt')
    transport_path = os.path.join(full_path,'tran.dat')


    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path)

    return species_list, reaction_list

In [16]:
CH3F_species, CH3F_reactions = load_chemkin_file_RMG('CH3F')
CH2F2_species, CH2F2_reactions = load_chemkin_file_RMG('CH2F2')

common_species, unique_species_CH3F, unique_species_CH2F2 = compare_species(CH3F_species, CH2F2_species)


In [17]:
#get names of unique species in CH3F mechanism without the index thats in the CH3F mechanism, and remove the S species bc thats hard to look up

CH3F_un_sp = [x.to_chemkin() for x in unique_species_CH3F]

import re
CH3F_un_sp_without_index = []
for spec in CH3F_un_sp:
    match = re.match('(\S+\()[0-9]+\)', spec)
    if match: 
        CH3F_un_sp_without_index.append(match.group(1))

CH3F_un_sp_without_index_no_S = ['Ar(',
 'He(',
 'C(T)(',  #checked
 'CH(',    #checked
 'CH3(',   #checked
 'C2H(',   #checked
 'C2H2(',  #checked
 'H2CC(',  #checked
 'CH3OH(', #checked
 'CH3O(',  #checked
 'C2H3(',  #checked
 'C2H4(',  #checked
 'C2H6(',  #checked
 'C2H5(',  #checked
 'CH2OH(', #checked
 'CH3CO(', #checked
 'CH2CHO(',#not showing up
 'CH3CHO(', #not showing up
 'CH2CHF(', #not showing up
#  'C2H4F(', #checked
 'C2H4F(', #checked
 'CH2CF(', #not showing up
 'C2H2F(', #checked
 'CH2CFO(', #not showing up
 'CH2FO(', #not showing up
 'CH2O2(', #checked
 'C2H2O(',#checked
#  'CFO3(', #cheked 
 'CFO3(']

In [18]:
CH2F2_un_sp = [x.to_chemkin() for x in unique_species_CH2F2]


import re
CH2F2_un_sp_without_index = []
for spec in CH2F2_un_sp:
    match = re.match('(\S+\()[0-9]+\)', spec)
    if match: 
        CH2F2_un_sp_without_index.append(match.group(1))
        
CH2F2_un_sp_without_index_no_S = ['CHFCF2(',
 'CH2CF2(',
#  'C2HF(',
 'CF2CF2(',
#  'C2H3F2(',
 'C2H3F2(',
 'C2H2F3(',
#  'C2H2F3(',
 'C2HF4(',
 'C2HF2(',
 'CF2CH(',
 'F2(',
 'CF2O2(',
#  'CF2O2(',
 'C2HF3(',
#  'CF2O2(',
 'CHFO(',
 'C2F(',
 'C2HF(']

See if there are any reactions with species that interact that are unique to the two pure halocarbon mechanisms  

In [19]:
import pandas

In [ ]:
df = pd.read_csv('/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core_redo/sensitivity/CH2F2_CH3F_RMG_sensitivities_2nd_time.csv')

Below is for finding cross reactions of 1 -> 2 or vice versa

In [48]:
#reactants to products 


#CH2F2 unique species in reactant
products_of_reactants_with_CH2F2_unique_species = []
for index, row in df.iterrows():
    for spec in CH2F2_un_sp_without_index_no_S:
        if f"'{spec}" in row['cantera reactants']:
            products_of_reactants_with_CH2F2_unique_species.append((row['cantera reactants'],row['cantera products'],index))
# reactant_cross_reactions = []
# for equation, ind in equations_with_CH2F2_unique_species:
#     for spec in CH3F_un_sp_without_index_no_S:
#         if f"'{spec}" in equation:
#             reactant_cross_reactions.append((equation, ind))

reactant_indexes_1=[]
reactant_cross_reactions_1 = []
for reactants, products,ind in products_of_reactants_with_CH2F2_unique_species:
    for spec in CH3F_un_sp_without_index_no_S:

        if f"'{spec}" in products:
            reactant_cross_reactions_1.append((reactants, products, ind))
            reactant_indexes_1.append((ind))

            
#CH3F unique species in reactant        
products_of_reactants_with_CH3F_unique_species = []
for index, row in df.iterrows():
    for spec in CH3F_un_sp_without_index_no_S:
        if f"'{spec}" in row['cantera reactants']:
            products_of_reactants_with_CH3F_unique_species.append((row['cantera reactants'],row['cantera products'],index))
# reactant_cross_reactions = []
# for equation, ind in equations_with_CH2F2_unique_species:
#     for spec in CH3F_un_sp_without_index_no_S:
#         if f"'{spec}" in equation:
#             reactant_cross_reactions.append((equation, ind))

reactant_indexes_2=[]
reactant_cross_reactions_2 = []
for reactants, products,ind in products_of_reactants_with_CH3F_unique_species:
    for spec in CH2F2_un_sp_without_index_no_S:

        if f"'{spec}" in products:
            reactant_cross_reactions_2.append((reactants, products, ind))
            reactant_indexes_2.append((ind))
            


In [96]:
big_list_of_indexes = reactant_indexes_1+reactant_indexes_2

In [102]:
sorted_list_of_indexes = sorted(big_list_of_indexes)
sorted_list_of_indexes

[88,
 112,
 115,
 145,
 224,
 253,
 273,
 279,
 288,
 307,
 309,
 323,
 332,
 350,
 351,
 368,
 440,
 458,
 469,
 471,
 479,
 498,
 498,
 504,
 504,
 518,
 529,
 533,
 551,
 566,
 575,
 575,
 581,
 584,
 599,
 611,
 616,
 645,
 684,
 689,
 689,
 695,
 700,
 708,
 708,
 718,
 730,
 735,
 735,
 742,
 744,
 749,
 752,
 761,
 766,
 768,
 769,
 772,
 777,
 792,
 800,
 813,
 824,
 825,
 825,
 847,
 852,
 852,
 855,
 855,
 870,
 883,
 889,
 892,
 900,
 904,
 906,
 923,
 924,
 927,
 927,
 952,
 955,
 970,
 979,
 984,
 992,
 992,
 995,
 1013,
 1029,
 1032,
 1057,
 1057,
 1062,
 1069,
 1070,
 1071,
 1075,
 1075,
 1083,
 1093,
 1096,
 1115,
 1115,
 1127,
 1137,
 1140,
 1140,
 1143,
 1147,
 1164,
 1165,
 1166,
 1167,
 1168,
 1172,
 1178,
 1183,
 1193,
 1210,
 1213,
 1223,
 1223,
 1224,
 1240,
 1268,
 1275,
 1291,
 1293,
 1294,
 1296,
 1304,
 1308,
 1308,
 1309,
 1310,
 1313,
 1330,
 1332,
 1337,
 1338,
 1354,
 1359,
 1378,
 1379,
 1399,
 1401,
 1401,
 1412,
 1412,
 1418,
 1420,
 1446,
 1449,
 1455

In [49]:
len(reactant_cross_reactions_1)+len(reactant_cross_reactions_2)

464

In [95]:
(set(reactant_cross_reactions_1+reactant_cross_reactions_2))

{("{'C(T)(87)': 1.0, 'C2H2F3(40)': 1.0}",
  "{'CF(31)': 1.0, 'CH2CF2(34)': 1.0}",
  2003),
 ("{'C(T)(87)': 1.0, 'C2H2F3(77)': 1.0}",
  "{'CF(31)': 1.0, 'CH2CF2(34)': 1.0}",
  1399),
 ("{'C(T)(87)': 1.0, 'C2HF2(42)': 1.0}",
  "{'C2HF(35)': 1.0, 'CF(31)': 1.0}",
  1523),
 ("{'C(T)(87)': 1.0, 'C2HF4(41)': 1.0}",
  "{'CF(31)': 1.0, 'CHFCF2(33)': 1.0}",
  2027),
 ("{'C(T)(87)': 1.0, 'CF2CH(43)': 1.0}",
  "{'C2HF(35)': 1.0, 'CF(31)': 1.0}",
  1137),
 ("{'C(T)(87)': 1.0, 'CHF2O(56)': 1.0}",
  "{'CF(31)': 1.0, 'CHFO(28)': 1.0}",
  2263),
 ("{'C(T)(87)': 1.0, 'CHFCF2(33)': 1.0}",
  "{'CF(31)': 1.0, 'CF2CH(43)': 1.0}",
  2677),
 ("{'C2F(84)': 1.0, 'C2H2F(107)': 1.0}", "{'C2HF(35)': 2.0}", 1294),
 ("{'C2F(84)': 1.0, 'C2H2O(112)': 1.0}",
  "{'C2HF(35)': 1.0, 'HCCO(18)': 1.0}",
  2434),
 ("{'C2F(84)': 1.0, 'C2H3(95)': 1.0}",
  "{'C2H2(91)': 1.0, 'C2HF(35)': 1.0}",
  1944),
 ("{'C2F(84)': 1.0, 'C2H3F(103)': 1.0}",
  "{'C2H2F(107)': 1.0, 'C2HF(35)': 1.0}",
  1083),
 ("{'C2F(84)': 1.0, 'C2H3F(103)': 1

In [51]:
_1_to_2_indexes = reactant_indexes_1+reactant_indexes_1

In [54]:
mega_set_1_goes_to_2 = reactant_cross_reactions_1+reactant_cross_reactions_2

Below was for finding cross reactions of 1 + 2 -> ....

In [66]:
#reactants

equations_with_CH2F2_unique_species = []
for index, row in df.iterrows():
    for spec in CH2F2_un_sp_without_index_no_S:
        if f"'{spec}" in row['cantera reactants']:
            equations_with_CH2F2_unique_species.append((row['cantera reactants'],index))
# reactant_cross_reactions = []
# for equation, ind in equations_with_CH2F2_unique_species:
#     for spec in CH3F_un_sp_without_index_no_S:
#         if f"'{spec}" in equation:
#             reactant_cross_reactions.append((equation, ind))

reactant_indexes=[]
reactant_cross_reactions = []
for equation, ind in equations_with_CH2F2_unique_species:
    for spec in CH3F_un_sp_without_index_no_S:

        if f"'{spec}" in equation:
            reactant_cross_reactions.append((equation, ind))
            reactant_indexes.append((ind))


In [67]:
len(reactant_indexes)

168

In [68]:
reactant_cross_reactions

[("{'CH3(89)': 1.0, 'CHFO(28)': 1.0}", 327),
 ("{'C2HF4(41)': 1.0, 'CH3(89)': 1.0}", 350),
 ("{'CF2CH(43)': 1.0, 'CH3(89)': 1.0}", 471),
 ("{'CF2CH(43)': 1.0, 'CH2CF(106)': 1.0}", 504),
 ("{'C2H2F(107)': 1.0, 'CF2CH(43)': 1.0}", 575),
 ("{'C2H2F3(77)': 1.0, 'CH2CF(106)': 1.0}", 584),
 ("{'CH2CF(106)': 1.0, 'CHFO(28)': 1.0}", 587),
 ("{'C2H2F(107)': 1.0, 'CHFO(28)': 1.0}", 591),
 ("{'C2H2F3(77)': 1.0, 'CH3(89)': 1.0}", 695),
 ("{'C2H3F2(37)': 1.0, 'CH3(89)': 1.0}", 755),
 ("{'C2H(90)': 1.0, 'CF2CH(43)': 1.0}", 777),
 ("{'C2HF4(41)': 1.0, 'CH2CF(106)': 1.0}", 825),
 ("{'CF2CH(43)': 1.0, 'CH(88)': 1.0}", 847),
 ("{'C2H2F(107)': 1.0, 'C2HF2(42)': 1.0}", 852),
 ("{'C2HF2(42)': 1.0, 'CH2CF(106)': 1.0}", 855),
 ("{'C2HF4(41)': 1.0, 'CH2CF(106)': 1.0}", 892),
 ("{'C2H(90)': 1.0, 'C2H2F3(77)': 1.0}", 927),
 ("{'C2H2F(107)': 1.0, 'C2H2F3(39)': 1.0}", 995),
 ("{'CH2O2(111)': 1.0, 'CHFO(28)': 1.0}", 1027),
 ("{'C2H3(95)': 1.0, 'CF2CH(43)': 1.0}", 1032),
 ("{'C2H2F(107)': 1.0, 'CF2CH(43)': 1.0}", 1

In [69]:
#products

equations_with_CH2F2_unique_species = []
for index, row in df.iterrows():
    for spec in CH2F2_un_sp_without_index_no_S:
        if f"'{spec}" in row['cantera products']:
            equations_with_CH2F2_unique_species.append((row['cantera products'],index))
            
product_indexes = []
product_cross_reactions = []
for equation, ind in equations_with_CH2F2_unique_species:
    for spec in CH3F_un_sp_without_index_no_S:
        if f"'{spec}" in equation:
            product_cross_reactions.append((equation, ind))
            product_indexes.append(ind)


In [70]:
product_cross_reactions

[("{'C2H2(91)': 1.0, 'CHFCF2(33)': 1.0}", 575),
 ("{'C2H2(91)': 1.0, 'CHFO(28)': 1.0}", 599),
 ("{'C2H2(91)': 1.0, 'CH2CF2(34)': 1.0}", 689),
 ("{'C2H2(91)': 1.0, 'F2(44)': 1.0}", 747),
 ("{'C2HF(35)': 1.0, 'CH3(89)': 1.0}", 766),
 ("{'CH2CF2(34)': 1.0, 'CH3(89)': 1.0}", 772),
 ("{'C2H2(91)': 1.0, 'CHFCF2(33)': 1.0}", 852),
 ("{'C2HF(35)': 1.0, 'CH2CF(106)': 1.0}", 904),
 ("{'C2H2F(107)': 1.0, 'CH2CF2(34)': 1.0}", 955),
 ("{'C2H2(91)': 1.0, 'C2HF(35)': 1.0}", 992),
 ("{'C2H2F(107)': 1.0, 'C2HF(35)': 1.0}", 1083),
 ("{'CH2CF(106)': 1.0, 'CH2CF2(34)': 1.0}", 1093),
 ("{'C2HF(35)': 1.0, 'CH3(89)': 1.0}", 1275),
 ("{'CH2CF2(34)': 1.0, 'CH3(89)': 1.0}", 1296),
 ("{'CH3(89)': 1.0, 'CHFO(28)': 1.0}", 1298),
 ("{'C2H4(96)': 1.0, 'CHFCF2(33)': 1.0}", 1308),
 ("{'C2H4(96)': 1.0, 'CHFO(28)': 1.0}", 1309),
 ("{'C2HF(35)': 1.0, 'CH3(89)': 1.0}", 1330),
 ("{'C2H4(96)': 1.0, 'CH2CF2(34)': 1.0}", 1401),
 ("{'C2H2(91)': 1.0, 'CF2CH(43)': 1.0}", 1455),
 ("{'C2H2(91)': 1.0, 'C2HF(35)': 1.0}", 1490),
 ("{

In [71]:
#make product_cross_reactions and reactant_cross_reactions sets so they cant have dups

product_cross_reactions_set = set(product_cross_reactions)
reactant_cross_reactions_set = set(reactant_cross_reactions)


product_indexes_set = set(product_indexes)
reactant_indexes_set = set(reactant_indexes)

In [72]:
mega_set = set(product_indexes + reactant_indexes)
mega_set_reactions = set(product_cross_reactions + reactant_cross_reactions)

In [75]:
mega_set #this is the sensitivity index of all the cross reactions

{327,
 350,
 471,
 504,
 575,
 584,
 587,
 591,
 599,
 689,
 695,
 747,
 755,
 766,
 772,
 777,
 825,
 847,
 852,
 855,
 892,
 904,
 927,
 955,
 992,
 995,
 1027,
 1032,
 1057,
 1069,
 1083,
 1093,
 1096,
 1127,
 1137,
 1140,
 1143,
 1164,
 1166,
 1167,
 1172,
 1183,
 1213,
 1224,
 1240,
 1268,
 1275,
 1294,
 1296,
 1298,
 1308,
 1309,
 1310,
 1330,
 1332,
 1338,
 1354,
 1359,
 1379,
 1399,
 1401,
 1418,
 1420,
 1446,
 1455,
 1464,
 1466,
 1490,
 1498,
 1511,
 1512,
 1523,
 1541,
 1550,
 1552,
 1558,
 1561,
 1573,
 1575,
 1579,
 1586,
 1594,
 1599,
 1607,
 1611,
 1623,
 1624,
 1633,
 1651,
 1653,
 1654,
 1659,
 1663,
 1667,
 1669,
 1697,
 1724,
 1729,
 1744,
 1753,
 1754,
 1774,
 1776,
 1781,
 1783,
 1809,
 1810,
 1817,
 1819,
 1820,
 1851,
 1853,
 1854,
 1855,
 1859,
 1860,
 1864,
 1867,
 1872,
 1873,
 1884,
 1886,
 1895,
 1906,
 1909,
 1924,
 1927,
 1934,
 1944,
 1946,
 1949,
 1958,
 1963,
 1980,
 1983,
 1996,
 2003,
 2013,
 2018,
 2019,
 2023,
 2027,
 2037,
 2049,
 2061,
 2063,
 206

In [76]:
mega_set_reactions_just_dict = [x[0] for x in mega_set_reactions]

In [88]:
len(mega_set_reactions_just_dict)

289

Flip the cross reactions and rerun flamespeed

In [78]:
#for 1+2 -> ...

#find rmg match to cantera reaction in sensitivity file
rmg_indexes_of_cross_reactions = []

for cross_rxn in mega_set_reactions_just_dict:
    
    prod_df = df.loc[df['cantera products'] == cross_rxn]
    react_df = df.loc[df['cantera reactants'] == cross_rxn]
    
    cross_rxn_rows = prod_df.append(react_df)
        
    for index, row in cross_rxn_rows.iterrows():
        ct_index, sensitivity, ct_equation, ct_products, ct_reactants = row['k_s'], row['sensitivity'], row['cantera equation'], eval(row['cantera products']), eval(row['cantera reactants'])

        #list of all products in each reaction 
        list_of_products = [(k+',')*int(v) for k,v in ct_products.items()]   #use ',' as a marker for distinction between different species
        products = set(list(filter(None, (''.join(list_of_products)).split(',')))) #now make all the products into a list. If two of the same species are listed as reactants, they should appear as two entries

        #list of all reactants in each reaction 
        list_of_reactants = [(k+',')*int(v) for k,v in ct_reactants.items()]   #use ',' as a marker for distinction between different species
        reactants= set(list(filter(None, (''.join(list_of_reactants)).split(',')))) #now make all the products into a list. If two of the same species are listed as reactants, they should appear as two entries


        #iterate through rmg reactions and find the match to the cantera reaction
        for rmg_rxn in RMG_reaction_list: 
                rmg_reactants = set([str(i.to_chemkin()) for i in rmg_rxn.reactants])
                rmg_products = set([str(i.to_chemkin()) for i in rmg_rxn.products])

                if (reactants==rmg_reactants) and (products==rmg_products) and ('M' in ct_equation) and ('M' in str(rmg_rxn)):
                    rmg_indexes_of_cross_reactions.append((rmg_rxn.index, str(rmg_rxn),ct_equation))
                    #print(ct_index,'appended')
                    continue
                if (reactants==rmg_reactants) and (products==rmg_products) and ('M' not in ct_equation) and ('M' not in str(rmg_rxn)):
                    rmg_indexes_of_cross_reactions.append((rmg_rxn.index, str(rmg_rxn),ct_equation))
                    continue

                if (reactants==rmg_reactants) and (products==rmg_products)and ('M' in ct_equation) and ('M' not in str(rmg_rxn)) :
                    rmg_indexes_of_cross_reactions.append((rmg_rxn.index, str(rmg_rxn),ct_equation))
                    continue


rmg_indexes_of_cross_reactions 


[(2038,
  'CH4(4) + CF2CH(43) <=> CH3(89) + CH2CF2(34)',
  'CF2CH(43) + CH4(4) <=> CH2CF2(34) + CH3(89)'),
 (2036,
  'CH2(T)(16) + C2H3F2(38) <=> CH3(89) + CH2CF2(34)',
  'C2H3F2(38) + CH2(T)(16) <=> CH2CF2(34) + CH3(89)'),
 (1742,
  'CH3F(22) + CH2CF(106) <=> CH3(89) + CH2CF2(34)',
  'CH2CF(106) + CH3F(22) <=> CH2CF2(34) + CH3(89)'),
 (2037,
  'CH2(T)(16) + C2H3F2(37) <=> CH3(89) + CH2CF2(34)',
  'C2H3F2(37) + CH2(T)(16) <=> CH2CF2(34) + CH3(89)'),
 (2239,
  'C2H(90) + C2H4F(104) <=> C2HF(35) + C2H4(96)',
  'C2H(90) + C2H4F(104) <=> C2H4(96) + C2HF(35)'),
 (2238,
  'C2H2F(107) + C2H3(95) <=> C2HF(35) + C2H4(96)',
  'C2H2F(107) + C2H3(95) <=> C2H4(96) + C2HF(35)'),
 (2237,
  'CH2CF(106) + C2H3(95) <=> C2HF(35) + C2H4(96)',
  'C2H3(95) + CH2CF(106) <=> C2H4(96) + C2HF(35)'),
 (2236,
  'C2F(84) + C2H5(98) <=> C2HF(35) + C2H4(96)',
  'C2F(84) + C2H5(98) <=> C2H4(96) + C2HF(35)'),
 (2231,
  'CF2CH(43) + C2H3F(103) <=> CH2CF(106) + CH2CF2(34)',
  'C2H3F(103) + CF2CH(43) <=> CH2CF(106) + CH2

In [81]:
rmg_indices_to_flip_ = [x[0] for x in rmg_indexes_of_cross_reactions ]

In [82]:
rmg_indices_to_flip = set(rmg_indices_to_flip_)

In [89]:
len(rmg_indices_to_flip)

246

In [63]:
#for 1 -> 2

#find rmg match to cantera reaction in sensitivity file
rmg_indexes_of_cross_reactions_1_to_2 = []

for reactant_dict, product_dict, sens_ind in mega_set_1_goes_to_2:
    
    cross_rxn_rows = df.loc[(df['cantera products']==product_dict) & (df['cantera reactants']==reactant_dict)]
#     react_df = df.loc[df['cantera reactants'] == reactant_dict]
    
#     cross_rxn_rows = prod_df.append(react_df)
        
    for index, row in cross_rxn_rows.iterrows():
        ct_index, sensitivity, ct_equation, ct_products, ct_reactants = row['k_s'], row['sensitivity'], row['cantera equation'], eval(row['cantera products']), eval(row['cantera reactants'])

        #list of all products in each reaction 
        list_of_products = [(k+',')*int(v) for k,v in ct_products.items()]   #use ',' as a marker for distinction between different species
        products = set(list(filter(None, (''.join(list_of_products)).split(',')))) #now make all the products into a list. If two of the same species are listed as reactants, they should appear as two entries

        #list of all reactants in each reaction 
        list_of_reactants = [(k+',')*int(v) for k,v in ct_reactants.items()]   #use ',' as a marker for distinction between different species
        reactants= set(list(filter(None, (''.join(list_of_reactants)).split(',')))) #now make all the products into a list. If two of the same species are listed as reactants, they should appear as two entries


        #iterate through rmg reactions and find the match to the cantera reaction
        for rmg_rxn in RMG_reaction_list: 
                rmg_reactants = set([str(i.to_chemkin()) for i in rmg_rxn.reactants])
                rmg_products = set([str(i.to_chemkin()) for i in rmg_rxn.products])

                if (reactants==rmg_reactants) and (products==rmg_products) and ('M' in ct_equation) and ('M' in str(rmg_rxn)):
                    rmg_indexes_of_cross_reactions.append((rmg_rxn.index, str(rmg_rxn),ct_equation))
                    #print(ct_index,'appended')
                    continue
                if (reactants==rmg_reactants) and (products==rmg_products) and ('M' not in ct_equation) and ('M' not in str(rmg_rxn)):
                    rmg_indexes_of_cross_reactions.append((rmg_rxn.index, str(rmg_rxn),ct_equation))
                    continue

                if (reactants==rmg_reactants) and (products==rmg_products)and ('M' in ct_equation) and ('M' not in str(rmg_rxn)) :
                    rmg_indexes_of_cross_reactions.append((rmg_rxn.index, str(rmg_rxn),ct_equation))
                    continue


rmg_indexes_of_cross_reactions 


[(2403,
  'OH(5) + CH2CF2(34) <=> CF2-2(119) + CH2OH(99)',
  'CH2CF2(34) + OH(5) <=> CF2-2(119) + CH2OH(99)'),
 (648, 'H(6) + CHFO(28) <=> CH2FO(109)', 'CHFO(28) + H(6) <=> CH2FO(109)'),
 (262,
  'H(6) + C2HF(35) <=> CH2CF(106)',
  'C2HF(35) + H(6) (+M) <=> CH2CF(106) (+M)'),
 (2143,
  'CHF2(45) + C2HF(35) <=> CF2-2(119) + CH2CF(106)',
  'C2HF(35) + CHF2(45) <=> CF2-2(119) + CH2CF(106)'),
 (263,
  'H(6) + C2HF(35) <=> C2H2F(107)',
  'C2HF(35) + H(6) (+M) <=> C2H2F(107) (+M)'),
 (1844,
  'CF2CH(43) + C2H2F(107) <=> CHFCF2(33) + C2H2(91)',
  'C2H2F(107) + CF2CH(43) <=> C2H2(91) + CHFCF2(33)'),
 (2320,
  'CHF2(45) + C2HF(35) <=> CF2-2(119) + C2H2F(107)',
  'C2HF(35) + CHF2(45) <=> C2H2F(107) + CF2-2(119)'),
 (1580,
  'CHFO(28) + CHFO2(49) <=> CFO3(113) + CH2F(27)',
  'CHFO(28) + CHFO2(49) <=> CFO3(113) + CH2F(27)'),
 (2115,
  'H(6) + C2HF(35) <=> HF(21) + C2H(90)',
  'C2HF(35) + H(6) <=> C2H(90) + HF(21)'),
 (2984,
  'CH4(4) + C2H2F3(77) <=> CH3(89) + S(476)',
  'C2H2F3(77) + CH4(4) <=> C

In [64]:
rmg_indexes_of_cross_reactions_ = [x[0] for x in rmg_indexes_of_cross_reactions ]

# Deleting Cross Reactions

In [226]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
#import numpy as np
import subprocess
import csv
#import scipy
import copy
import os

In [90]:
rmg_indices = set(rmg_indexes_of_cross_reactions_ + rmg_indices_to_flip_) 
#all cross reactions, whether 1-> 2 or 1+2-> ...

In [94]:
############################### convert the indicated reactions to use NIST kinetics ##########################################
new_reaction_list = [r for r in RMG_reaction_list if r.index not in rmg_indices ]
new_species_list = RMG_species_list
#save the chemkin file 

handpicked_dir = '/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core_redo/CH2F2_CH3F/chemkin/'


chemkin_file = os.path.join(handpicked_dir, f'cross_rxns_deleted_updated.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, new_species_list, new_reaction_list, verbose=True, check_for_duplicates=True)
